In [1]:
!pip install order_of_magnitude
!pip install netCDF4

  Preparing metadata (setup.py) ... done
  Created wheel for order_of_magnitude: filename=order_of_magnitude-2.3.1-py3-none-any.whl size=4579 sha256=6a58da6e404b9cea52f71de6df47fd298370b68b1583d3ebe2eef91cb763b729
  Stored in directory: /root/.cache/pip/wheels/85/53/9b/310924c3bd35596e70e6d3597f347986a2f95cfb6393e374ec
Successfully built order_of_magnitude
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.8 MB/s eta 0:00:00


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("drive/My Drive/ERA5_Dataset")

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import netCDF4 as nc
import util

## Target Dataframe

In [4]:
df_target = pd.read_csv('./csv/target_NoOutliers_complete.csv')

lats = df_target['LAT'].values
lons = df_target['LON'].values

df_target

,DATE,ID,LAT,LON,S.IndAll
0,1980-01-01,NaN,-18.50,80.50,0.00
1,1980-01-02,NaN,-30.00,31.25,0.00
2,1980-01-03,NaN,-4.00,86.50,0.00
3,1980-01-04,NaN,-29.50,32.75,0.00
4,1980-01-05,NaN,-4.75,89.25,0.00
...,...,...,...,...,...
15422,2022-03-23,392.0,-13.75,76.75,0.50
15423,2022-03-24,392.0,-14.25,74.25,1.00
15424,2022-03-25,392.0,-13.50,73.25,1.00
15425,2022-03-26,392.0,-15.00,74.00,1.00


In [6]:
N_dates = df_target.shape[0]
print(N_dates)

15427


## Sea Surface Level Temperature

In [7]:
ds_prec = nc.Dataset('./nc_original_datasets/precipitation.nc')
prec = ds_prec['tp'][ : , 0, :, :]

In [8]:
prec_zones = []

prec_zones.append(prec[:N_dates, 0:30, 0:60])
prec_zones.append(prec[:N_dates, 0:30, 60:120])
prec_zones.append(prec[:N_dates, 0:30, 120:180])
prec_zones.append(prec[:N_dates, 0:30, 180:])

prec_zones.append(prec[:N_dates, 30:60, 0:60])
prec_zones.append(prec[:N_dates, 30:60, 60:120])
prec_zones.append(prec[:N_dates, 30:60, 120:180])
prec_zones.append(prec[:N_dates, 30:60, 180:])

prec_zones.append(prec[:N_dates, 60:90, 0:60])
prec_zones.append(prec[:N_dates, 60:90, 60:120])
prec_zones.append(prec[:N_dates, 60:90, 120:180])
prec_zones.append(prec[:N_dates, 60:90, 180:])

prec_zones.append(prec[:N_dates, 90:, 0:60])
prec_zones.append(prec[:N_dates, 90:, 60:120])
prec_zones.append(prec[:N_dates, 90:, 120:180])
prec_zones.append(prec[:N_dates, 90:, 180:])

In [9]:
for i in range(16):
  print(prec_zones[i].shape)

(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 61)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 61)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 60)
(15427, 30, 61)
(15427, 31, 60)
(15427, 31, 60)
(15427, 31, 60)
(15427, 31, 61)


In [11]:
mean_zones = []
std_zones = []

for zone in range(16):
  means = []
  stds = []
  for date_idx in range(N_dates):
    temp_mean = np.mean(prec_zones[zone][date_idx])
    temp_std = np.std(prec_zones[zone][date_idx])
    means.append(temp_mean)
    stds.append(temp_std)
  mean_zones.append(means)
  std_zones.append(stds)

In [12]:
mean_std_zones = np.column_stack((mean_zones[0], std_zones[0],
                                  mean_zones[1], std_zones[1],
                                  mean_zones[2], std_zones[2],
                                  mean_zones[3], std_zones[3],
                                  mean_zones[4], std_zones[4],
                                  mean_zones[5], std_zones[5],
                                  mean_zones[6], std_zones[6],
                                  mean_zones[7], std_zones[7],
                                  mean_zones[8], std_zones[8],
                                  mean_zones[9], std_zones[9],
                                  mean_zones[10], std_zones[10],
                                  mean_zones[11], std_zones[11],
                                  mean_zones[12], std_zones[12],
                                  mean_zones[13], std_zones[13],
                                  mean_zones[14], std_zones[14],
                                  mean_zones[15], std_zones[15]))

# Save Datset

In [13]:
target_att = ['DATE', 'S.IndAll']
mean_std_zones_att = ['Mean_Z1', 'Std_Z1',
                      'Mean_Z2', 'Std_Z2',
                      'Mean_Z3', 'Std_Z3',
                      'Mean_Z4', 'Std_Z4',
                      'Mean_Z5', 'Std_Z5',
                      'Mean_Z6', 'Std_Z6',
                      'Mean_Z7', 'Std_Z7',
                      'Mean_Z8', 'Std_Z8',
                      'Mean_Z9', 'Std_Z9',
                      'Mean_Z10', 'Std_Z10',
                      'Mean_Z11', 'Std_Z11',
                      'Mean_Z12', 'Std_Z12',
                      'Mean_Z13', 'Std_Z13',
                      'Mean_Z14', 'Std_Z14',
                      'Mean_Z15', 'Std_Z15',
                      'Mean_Z16', 'Std_Z16']

target = df_target[target_att].values
final_array = np.hstack((target, mean_std_zones))

df_complete = pd.DataFrame(final_array, columns=target_att+mean_std_zones_att)
df_complete[11:]

,DATE,S.IndAll,Mean_Z1,Std_Z1,Mean_Z2,Std_Z2,Mean_Z3,Std_Z3,Mean_Z4,Std_Z4,...,Mean_Z12,Std_Z12,Mean_Z13,Std_Z13,Mean_Z14,Std_Z14,Mean_Z15,Std_Z15,Mean_Z16,Std_Z16
11,1980-01-12,0.0,0.000166,0.000525,0.000123,0.000291,0.000177,0.000362,0.000488,0.000718,...,0.000064,0.000104,0.000017,0.000044,0.00022,0.000536,0.000171,0.000325,0.000056,0.00008
12,1980-01-13,0.0,0.000106,0.00035,0.000035,0.000049,0.000091,0.000097,0.000515,0.000826,...,0.000073,0.000073,0.000015,0.000046,0.000391,0.001534,0.000112,0.000197,0.000057,0.000062
13,1980-01-14,0.0,0.000026,0.000104,0.000044,0.000056,0.000286,0.000573,0.00069,0.00092,...,0.000054,0.00006,0.000038,0.000107,0.000127,0.000235,0.00006,0.000099,0.000037,0.000052
14,1980-01-15,0.0,0.000051,0.000252,0.000038,0.000053,0.000474,0.000905,0.001055,0.001204,...,0.000115,0.000224,0.000021,0.000057,0.000302,0.000542,0.000085,0.000212,0.000037,0.000056
15,1980-01-16,0.0,0.000085,0.000365,0.000026,0.000045,0.000138,0.000226,0.000506,0.00079,...,0.000153,0.000264,0.000033,0.000051,0.000204,0.000443,0.000329,0.000753,0.000038,0.000054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15422,2022-03-23,0.5,0.000212,0.000615,0.000062,0.00008,0.000281,0.000436,0.000056,0.000073,...,0.000418,0.000666,0.00004,0.000071,0.000081,0.000126,0.000077,0.000151,0.000057,0.000065
15423,2022-03-24,1.0,0.000049,0.000143,0.000055,0.000071,0.000571,0.000845,0.000052,0.000102,...,0.000506,0.00117,0.000029,0.000061,0.00006,0.00008,0.000064,0.000097,0.000041,0.000057
15424,2022-03-25,1.0,0.000022,0.000077,0.00002,0.000031,0.000757,0.001138,0.000139,0.000537,...,0.000594,0.001489,0.000023,0.00008,0.000045,0.000053,0.000051,0.000051,0.000043,0.000061
15425,2022-03-26,1.0,0.000028,0.000146,0.000028,0.000043,0.000398,0.000633,0.000069,0.000205,...,0.000584,0.001259,0.000281,0.000715,0.000091,0.000179,0.000044,0.000055,0.000042,0.000075


# Rollback 10 days

In [14]:
def attribute_stepped(atts, max_steps):
  att_list = []
  for att in atts:
    for i in range(max_steps):
      att_list.append('{}-{}'.format(att,i))
  return att_list

In [15]:
def mean_std_atts_zones(zones_n):
  new_att_list = []
  for i in range(zones_n):
    new_att_list.append('Mean_Z{}'.format(i+1))
    new_att_list.append('Std_Z{}'.format(i+1))
  return new_att_list

print(mean_std_atts_zones(16))

['Mean_Z1', 'Std_Z1', 'Mean_Z2', 'Std_Z2', 'Mean_Z3', 'Std_Z3', 'Mean_Z4', 'Std_Z4', 'Mean_Z5', 'Std_Z5', 'Mean_Z6', 'Std_Z6', 'Mean_Z7', 'Std_Z7', 'Mean_Z8', 'Std_Z8', 'Mean_Z9', 'Std_Z9', 'Mean_Z10', 'Std_Z10', 'Mean_Z11', 'Std_Z11', 'Mean_Z12', 'Std_Z12', 'Mean_Z13', 'Std_Z13', 'Mean_Z14', 'Std_Z14', 'Mean_Z15', 'Std_Z15', 'Mean_Z16', 'Std_Z16']


In [16]:
max_steps = 11
labels_to_rewind = mean_std_atts_zones(16)
labels_stepped = attribute_stepped(labels_to_rewind, max_steps)
print(labels_stepped)

stepped_mean_zones = []
stepped_std_zones = []

for zone in range(1, 17):
  mean_press_10d = np.zeros((N_dates-max_steps, max_steps))
  std_press_10d = np.zeros((N_dates-max_steps, max_steps))

  mean_press = df_complete['Mean_Z{}'.format(zone)]
  std_press = df_complete['Std_Z{}'.format(zone)]

  for day in range(max_steps, N_dates):
    for steps in range(max_steps):
      mean_press_10d[day-max_steps, steps] = mean_press[day-steps]
      std_press_10d[day-max_steps, steps] = std_press[day-steps]
  print(mean_press_10d.shape)
  stepped_mean_zones.append(mean_press_10d)
  stepped_std_zones.append(std_press_10d)

['Mean_Z1-0', 'Mean_Z1-1', 'Mean_Z1-2', 'Mean_Z1-3', 'Mean_Z1-4', 'Mean_Z1-5', 'Mean_Z1-6', 'Mean_Z1-7', 'Mean_Z1-8', 'Mean_Z1-9', 'Mean_Z1-10', 'Std_Z1-0', 'Std_Z1-1', 'Std_Z1-2', 'Std_Z1-3', 'Std_Z1-4', 'Std_Z1-5', 'Std_Z1-6', 'Std_Z1-7', 'Std_Z1-8', 'Std_Z1-9', 'Std_Z1-10', 'Mean_Z2-0', 'Mean_Z2-1', 'Mean_Z2-2', 'Mean_Z2-3', 'Mean_Z2-4', 'Mean_Z2-5', 'Mean_Z2-6', 'Mean_Z2-7', 'Mean_Z2-8', 'Mean_Z2-9', 'Mean_Z2-10', 'Std_Z2-0', 'Std_Z2-1', 'Std_Z2-2', 'Std_Z2-3', 'Std_Z2-4', 'Std_Z2-5', 'Std_Z2-6', 'Std_Z2-7', 'Std_Z2-8', 'Std_Z2-9', 'Std_Z2-10', 'Mean_Z3-0', 'Mean_Z3-1', 'Mean_Z3-2', 'Mean_Z3-3', 'Mean_Z3-4', 'Mean_Z3-5', 'Mean_Z3-6', 'Mean_Z3-7', 'Mean_Z3-8', 'Mean_Z3-9', 'Mean_Z3-10', 'Std_Z3-0', 'Std_Z3-1', 'Std_Z3-2', 'Std_Z3-3', 'Std_Z3-4', 'Std_Z3-5', 'Std_Z3-6', 'Std_Z3-7', 'Std_Z3-8', 'Std_Z3-9', 'Std_Z3-10', 'Mean_Z4-0', 'Mean_Z4-1', 'Mean_Z4-2', 'Mean_Z4-3', 'Mean_Z4-4', 'Mean_Z4-5', 'Mean_Z4-6', 'Mean_Z4-7', 'Mean_Z4-8', 'Mean_Z4-9', 'Mean_Z4-10', 'Std_Z4-0', 'Std_Z4-1', 

In [17]:
final_array = np.hstack((target[max_steps:],
                         stepped_mean_zones[0], stepped_std_zones[0],
                         stepped_mean_zones[1], stepped_std_zones[1],
                         stepped_mean_zones[2], stepped_std_zones[2],
                         stepped_mean_zones[3], stepped_std_zones[3],
                         stepped_mean_zones[4], stepped_std_zones[4],
                         stepped_mean_zones[5], stepped_std_zones[5],
                         stepped_mean_zones[6], stepped_std_zones[6],
                         stepped_mean_zones[7], stepped_std_zones[7],
                         stepped_mean_zones[8], stepped_std_zones[8],
                         stepped_mean_zones[9], stepped_std_zones[9],
                         stepped_mean_zones[10], stepped_std_zones[10],
                         stepped_mean_zones[11], stepped_std_zones[11],
                         stepped_mean_zones[12], stepped_std_zones[12],
                         stepped_mean_zones[13], stepped_std_zones[13],
                         stepped_mean_zones[14], stepped_std_zones[14],
                         stepped_mean_zones[15], stepped_std_zones[15],
                         ))
df_stepped = pd.DataFrame(final_array , columns = target_att + labels_stepped)
df_stepped

,DATE,S.IndAll,Mean_Z1-0,Mean_Z1-1,Mean_Z1-2,Mean_Z1-3,Mean_Z1-4,Mean_Z1-5,Mean_Z1-6,Mean_Z1-7,...,Std_Z16-1,Std_Z16-2,Std_Z16-3,Std_Z16-4,Std_Z16-5,Std_Z16-6,Std_Z16-7,Std_Z16-8,Std_Z16-9,Std_Z16-10
0,1980-01-12,0.0,0.000166,0.000084,0.000027,0.000073,0.000042,0.000042,0.000049,0.000139,...,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052,0.000044,0.000057,0.000066,0.000088
1,1980-01-13,0.0,0.000106,0.000166,0.000084,0.000027,0.000073,0.000042,0.000042,0.000049,...,0.00008,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052,0.000044,0.000057,0.000066
2,1980-01-14,0.0,0.000026,0.000106,0.000166,0.000084,0.000027,0.000073,0.000042,0.000042,...,0.000062,0.00008,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052,0.000044,0.000057
3,1980-01-15,0.0,0.000051,0.000026,0.000106,0.000166,0.000084,0.000027,0.000073,0.000042,...,0.000052,0.000062,0.00008,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052,0.000044
4,1980-01-16,0.0,0.000085,0.000051,0.000026,0.000106,0.000166,0.000084,0.000027,0.000073,...,0.000056,0.000052,0.000062,0.00008,0.000118,0.000053,0.000057,0.000041,0.000042,0.000052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15411,2022-03-23,0.5,0.000212,0.000204,0.000018,0.000021,0.000017,0.000175,0.000252,0.000016,...,0.000079,0.000078,0.00004,0.000044,0.00007,0.000049,0.000047,0.00009,0.000064,0.000091
15412,2022-03-24,1.0,0.000049,0.000212,0.000204,0.000018,0.000021,0.000017,0.000175,0.000252,...,0.000065,0.000079,0.000078,0.00004,0.000044,0.00007,0.000049,0.000047,0.00009,0.000064
15413,2022-03-25,1.0,0.000022,0.000049,0.000212,0.000204,0.000018,0.000021,0.000017,0.000175,...,0.000057,0.000065,0.000079,0.000078,0.00004,0.000044,0.00007,0.000049,0.000047,0.00009
15414,2022-03-26,1.0,0.000028,0.000022,0.000049,0.000212,0.000204,0.000018,0.000021,0.000017,...,0.000061,0.000057,0.000065,0.000079,0.000078,0.00004,0.000044,0.00007,0.000049,0.000047


In [18]:
df_stepped.to_csv('./csv/16_zones/precipitation_10D.csv', index=False)